<a href="https://colab.research.google.com/github/solomontessema/building-agentic-ai/blob/main/notebooks/4.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table>
  <tr>
    <td><img src="https://ionnova.com/img/ionnova_logo_name_2.png" width="120px"></td>
    <td><h1>Prompt Flow Refactor</h1></td>
  </tr>
</table>

In [ ]:
!pip install -qU langchain==1.1.0 langchain-openai==1.1.0 python-dotenv==1.1.1

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-4", temperature=0)

# 1: Intent Classification Prompt
intent_prompt = PromptTemplate.from_template(
    """
    Classify the intent of this query: {query}
    Categories: weather, news, search, database, unknown.
    Just return the category.
    """
)

# Define the runnable for intent classification
intent_classifier = (
    # Ensure the input 'query' is passed to the prompt
    {"query": RunnablePassthrough()}
    | intent_prompt
    | llm
)
# 2: Generate Follow-up Prompt with Conditional Logic
followup_prompt = PromptTemplate(
    input_variables=["category", "constraint"],
    template="""
    Category: {{ category }}

    {% if constraint %}
    Please ensure to follow this rule: {{ constraint }}
    {% endif %}

    Write a follow-up question for clarification.
    """,
    template_format="jinja2"
)
# Define the full workflow using LCEL composition
clarifier = (
    # This structure maps inputs for the followup_prompt:
    {
        # 1. 'category' comes from the output of the intent_classifier (Step 1)
        "category": intent_classifier,
        # 2. 'constraint' comes directly from the original input
        "constraint": RunnablePassthrough.assign(category=intent_classifier).pick("constraint"),
    }
    # 3. The mapped dictionary is passed to the followup prompt and LLM
    | followup_prompt
    | llm
)

# Example Execution:
result = clarifier.invoke({
    "query": "Tell me something interesting about AI.",
    "constraint": "Keep it beginner-friendly."
})
print(result.content)
